# Assignment 1
Bradley Thompson - CS 510 LLM Winter 2024

## 1
Three differences between bloom and bloom z models:
1. Bloom seems to be more likely to generate output in first person, versus second or third person for bloomz.
2. Bloomz responses seem to be more dire / dark (e.g. "the world was in a state of war").
3. Bloomz is fine-tuned to a task mixture designed by the researchers.

## 2

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from typing import Dict, Any
import evaluate
import torch as t

DEFAULT_PROMPT = "Once upon a time "

perplexity = evaluate.load("perplexity", module_type="metric")

def load_and_gen(model_name: str, prompt: str, config: Dict[str, Any]):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    gen_config: GenerationConfig = GenerationConfig.from_dict(config)
    return tokenizer, model.generate(inputs, gen_config)

def calculate_metrics(model_name, tokenizer, output):
    print("Raw: ", output)
    input_text = tokenizer.decode(output)
    print("Decoded: ", input_text)
    ttr = len(t.unique(output)) / len(output)
    print("TTR: ", ttr)
    # https://stackoverflow.com/questions/75886674/how-to-compute-sentence-level-perplexity-from-hugging-face-language-models
    results = perplexity.compute(model_id=model_name, add_start_token=False, predictions=[input_text])
    print("Perplexity: ", results["perplexities"][0])
    

In [2]:
tokenizer_560m, outputs_560m = load_and_gen("bigscience/bloom-560m", DEFAULT_PROMPT, {
    "min_new_tokens": 100,
    "max_new_tokens": 200,
    "use_cache": False,
    "no_repeat_ngram_size": 2,
})
calculate_metrics("bigscience/bloom-560m", tokenizer_560m, outputs_560m[0])

Raw:  tensor([ 64393,  14591,    267,   3509,    210,    473,   1620,    267,   1517,
           461,    368,   8876,    189,   5024,    473,   3866,    267,  32531,
           530,    267,   2158,    189,   6475,    473,   4984,   1130,    722,
           267,  27782,    189,     44,   4984,   3804,    722,    368,   1517,
           189,   2175,   1517,   5268,   3866,    427,    722,    189,     36,
          1517,   1002,    267,  23724,    189,  11848,    267,  23037,    189,
          4670,    722,  11045,    427,  19134,    189,   6895,    861,    473,
          9746,    722,  88889,    189,   7594,    368,   6199,    189,  57647,
         88889,   1074,    189,  22654,  88889,   2670,  27782,    530,   2670,
         27432,    189,  10367,  27782,   1620,    368,   2592,   5268,  88889,
           368,   7733,    189,   5563,  88889,   3868,  32531,    189,  21449,
         88889,   6371,    189,  55857,  19134,   1620,   1427,  10087,    189,
         15466,   7154,   4984,  1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/1 [00:00<?, ?it/s]

Perplexity:  6.80005407333374


In [ ]:
tokenizer_1b1, outputs_1b1 = load_and_gen("bigscience/bloom-1b1", DEFAULT_PROMPT, {
    "min_new_tokens": 100,
    "max_new_tokens": 200,
    "no_repeat_ngram_size": 2,
})
calculate_metrics("bigscience/bloom-1b1", tokenizer_1b1, outputs_1b1[0])

In [ ]:
tokenizer_1b7, outputs_1b7 = load_and_gen("bigscience/bloom-1b7", DEFAULT_PROMPT, {
    "min_new_tokens": 100,
    "max_new_tokens": 200,
})
calculate_metrics("bigscience/bloom-1b7", tokenizer_1b7, outputs_1b7[0])

In [ ]:
tokenizer_z560m, outputs_z560m = load_and_gen("bigscience/bloomz-560m", DEFAULT_PROMPT, {
    "min_length": 100,
    "max_length": 200,
    "use_cache": False,
    "num_beams": 2, # Need to use beem sampling or else bloomz stops early
    "early_stopping": "never",
})
calculate_metrics("bigscience/bloomz-560m", tokenizer_z560m, outputs_z560m[0])

In [ ]:
tokenizer_z1b1, outputs_z1b1 = load_and_gen("bigscience/bloomz-1b1", DEFAULT_PROMPT, {
    "min_length": 100,
    "max_length": 200,
    "use_cache": False,
    "num_beams": 2,
    "early_stopping": "never",
})
calculate_metrics("bigscience/bloomz-560m", tokenizer_z1b1, outputs_z1b1[0])

In [ ]:
tokenizer_z1b7, outputs_z1b7 = load_and_gen("bigscience/bloomz-1b7", DEFAULT_PROMPT, {
    "min_length": 100,
    "max_length": 200,
    "use_cache": False,
    "num_beams": 2,
    "early_stopping": "never",
})
calculate_metrics("bigscience/bloomz-1b7", tokenizer_z1b7, outputs_z1b7[0])

### 2.a
The regular bloom models were easy-enough to interact with, but the bloomz models kept stopping early. I tried a lot of things to get it to stop using the EOS token, but no matter what it generated short output sequences. Eventually I found a way to do it when I found the 'early_stopping' config. When I set that to 'never', I got an error saying I needed to use beam sampling, so I set the beam count to 2 (5 originally but it was way too slow).

### 2.b
TTR and Perplexity metrics calculated inline in cell outputs above. In general all of the models have very low TTR, which is probably because of the limited textual input to build off of. Perplexity seems to be mildly higher in the bloomz models' outputs; however, this doesn't manifest in a noticeably different or less-repetitive generated sequence.

### 2.c
The generated stories are all pretty bad. For whatever reason, all models generate extremely repetitive sequences. The bloomz 1.1b parameter model seemed to have the most variety in that it alternated what phrases it output. In general, however, I'd say the creativity, coherence and overall quality of all model outputs were quite low.

## 3
Increasing model parameters seems to have a positive effect in that it lowers model perplexity. I think it would be better if we provided more context to perhaps see if the models would generate more diverse output. Type-to-token Ration seems relatively unaffected by the parameter size, so perhaps that is a metric that is affected more by the vocabulary for the model's training / pre-training data. Since the bloomz models are built off the bloom models, it makes sense that they have a similar vocabulary and similar TTR results.

## 4
I think that the two best models were the 560m and 1.1b parameter bloomz models. This is because of the mild improvement in vocublary diversity in the generated sequences. I tried experimenting with temperature, top_k and top_p decoding. It is really hard to get the model to avoid stopping early when not using beam sampling to explicitly tell it not to. I found that top_k sampling didn't have much effect on the repetitive behavior, which makes sense because the model is already clearly stuck on the same highly-probably tokens to choose from. Temperature seems to have a solid effect on diversifying the generated stories; it seems like injecting more randomness helps break out of the repetitive generation pattern. 

A mix of `temperature` and `top_p` decoding seem to be the most effective hyperparameters to tweak. In the bloomz-1b1 example below, I found that I was able to introduce an adequately lengthed sequence with some level of diversity in the content -- the phrases were not EXACTLY the same, repeated over and over, anymore! I started with a very low temperature of 0.3, then increased it to 0.8. I was a little surprised, thinking that more randomness would result in a greater diversity of content in the generated sequence. Conversely, less randomness from a higher temperature actually allowed the story to build off of itself. The bloomz-1b1 output at the bottom of this document shows a story that, while still not very good and a little repetitive, is actually a story!

In [ ]:
tokenizer_z560m, outputs_z560m = load_and_gen("bigscience/bloomz-560m", DEFAULT_PROMPT, {
    "min_length": 100,
    "max_length": 200,
    "use_cache": False,
    "do_sample": True,
    "temperature": 0.7,
    "top_p": 10,
})
calculate_metrics("bigscience/bloomz-560m", tokenizer_z560m, outputs_z560m[0])

In [ ]:
tokenizer_z1b1, outputs_z1b1 = load_and_gen("bigscience/bloomz-1b1", DEFAULT_PROMPT, {
    "min_length": 100,
    "max_length": 200,
    "use_cache": False,
    "do_sample": True,
    "temperature": 0.8,
    "top_p": 40,
})
calculate_metrics("bigscience/bloomz-560m", tokenizer_z1b1, outputs_z1b1[0])